In [2]:
%matplotlib inline
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

import sys, os

from lammpsrun import LAMMPS, Prism

from ase import Atoms, units
from ase.visualize import view
from ase.io.trajectory import Trajectory
from ase.io import write, read
from ase.neighborlist import neighbor_list
from ase.build import surface
from ase.spacegroup import crystal
from ase.geometry import *

from pymatgen.core.surface import *
from pymatgen.io.ase import AseAtomsAdaptor as AAA

hpc_path = "G:\\home\\LAMMPS_Simulation\\HPC_Jupyter\\"

# for running on Linux
# hpc_path = "/local/yiming/Desktop/yx6015/home/LAMMPS_Simulation/HPC_Jupyter/"

# print(homepath + '\n' + hpc_path)

Crystallographic data is obtained from https://materials.springer.com/isp/crystallographic/docs/sd_1628167

# Bulk siderite crystal

In [3]:
distorted_siderite = AAA.get_structure(read('fe_o_water_fe_o_c_full_water.pdb'))

In [4]:
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer

In [5]:
sg = SpacegroupAnalyzer(distorted_siderite, symprec = 0.01)

In [6]:
distorted_siderite = sg.get_conventional_standard_structure()

In [6]:
distorted_siderite.to(filename="fe_o_water_fe_o_c_full_water.cif")

# Surfaces/Slabs

pymatgen referenc: https://matgenb.materialsvirtuallab.org/2017/04/03/Slab-generation-and-Wulff-shape.html

In [20]:
def find_top_site(sites_list, atomstring):
    c_indices = []
    c_coord_z = []

    for i, site in enumerate(sites_list):
        if site.species_string == atomstring:
            c_indices.append(i)
            c_coord_z.append(site.z)

        # Ensuring sequence is sorted
        c_indices_z = sorted(zip(c_coord_z, c_indices))
        
    index_to_remove = c_indices_z[-1][1]
    
    return index_to_remove

In [28]:
def get_ase_slab(pmg_struct, hkl=(1, 1, 1), min_thick=10, min_vac=10, symmetrize=False):
    # Modified from http://henniggroup.github.io/MPInterfaces/_modules/mpinterfaces/utils.html
    
    from pymatgen.io.ase import AseAtomsAdaptor
    """
    takes in the intial structure as pymatgen Structure object
    uses ase to generate the slab
    returns pymatgen Slab object

    Args:
        pmg_struct: pymatgen structure object
        hkl: hkl index of surface of slab to be created
        min_thick: minimum thickness of slab in Angstroms
        min_vac: minimum vacuum spacing
    """
    ase_atoms = AseAtomsAdaptor().get_atoms(pmg_struct)
    pmg_slab_gen = SlabGenerator(pmg_struct, hkl, min_thick, min_vac)
    h = pmg_slab_gen._proj_height
    nlayers = int(math.ceil(pmg_slab_gen.min_slab_size / h))
    ase_slab = surface(ase_atoms, hkl, nlayers)
    ase_slab.center(vacuum=min_vac / 2, axis=2)
    pmg_slab_structure = AseAtomsAdaptor().get_structure(ase_slab)
    
    final_slab = Slab(lattice=pmg_slab_structure.lattice,
                      species=pmg_slab_structure.species_and_occu,
                      coords=pmg_slab_structure.frac_coords,
                      site_properties=pmg_slab_structure.site_properties,
                      miller_index=hkl, oriented_unit_cell=pmg_slab_structure,
                      shift=0., scale_factor=None, energy=None)
    if symmetrize:
        final_slab = pmg_slab_gen.nonstoichiometric_symmetrized_slab(final_slab)
    return final_slab

In [19]:
def gen_surfaces(layers):
    print("Layers:", layers)
    for key in siderite_surface.keys():
        miller_index = [int(x) for x in key]

        slabgen = SlabGenerator(siderite, miller_index,
                                min_slab_size=layers,
                                min_vacuum_size=10,
                                center_slab=True,
                                in_unit_planes=True)

        slab_list = slabgen.get_slabs(bonds={("C", "O"): 1.5},
                                      symmetrize=True)

        siderite_surface[key] = [x for x in slab_list if abs(x.charge) <= 4]

    surface_110_NS = siderite_surface['110'][2].copy()
    surface_110_NS.symmetrically_remove_atoms([find_top_site(surface_110_NS.sites, 'C4+')])
    surface_012_NS = siderite_surface['012'][2].copy()
    surface_012_NS.symmetrically_remove_atoms([find_top_site(surface_012_NS.sites, 'O2-')])
    surface_012_S = siderite_surface['012'][0].copy()
    surface_012_S_top_C = find_top_site(surface_012_S.sites, 'C4+')
    surface_012_S_top_O3 = surface_012_S.get_neighbors(surface_012_S.sites[surface_012_S_top_C],1.5, include_index=True)
    surface_012_S_top_CO3 = [surface_012_S_top_C] + [x[2] for x in surface_012_S_top_O3]
    surface_012_S.symmetrically_remove_atoms(surface_012_S_top_CO3)

    siderite_surface_kept = {}
    siderite_surface_kept['104_S'] = siderite_surface['104'][0].copy()
    siderite_surface_kept['110_S'] = siderite_surface['110'][1].copy()
    siderite_surface_kept['110_NS'] = surface_110_NS
    siderite_surface_kept['110_NS'] = siderite_surface['110'][0].copy()
    siderite_surface_kept['101_NS0'] = siderite_surface['101'][0].copy()
    siderite_surface_kept['101_NS1'] = siderite_surface['101'][1].copy()
    siderite_surface_kept['001_NS'] = siderite_surface['001'][0].copy()
    siderite_surface_kept['012_S'] = surface_012_S
    siderite_surface_kept['012_NS'] = surface_012_NS
    siderite_surface_kept['100_S'] = siderite_surface['100'][0].copy()

    directory_path = os.path.join(hpc_path, "siderite", "surfaces")
    for surface, slab in siderite_surface_kept.items():
        slab_ase = AAA.get_atoms(slab)
        layer_info, layer_distances = get_layers(slab_ase, (0,0,1), tolerance=0.1)
        nlayers = max(layer_info)

#         print("Surface:", surface, "\tLayers:", nlayers)
        file_path = os.path.join(directory_path, "surface{0}_L{1}.extxyz".format(surface, nlayers))
        slab_ase.write(file_path, format='extxyz')
    return siderite_surface_kept

Past work done with surface 104, 110, 100, 101, 001 and 012. High surface energy on 100 and 101

From bulk optimization we have new cell parameters.

In [9]:
siderite = distorted_siderite.copy()
siderite.add_oxidation_state_by_element({"Fe": 2, "C": 4, "O": -2})

In [10]:
siderite_surfaces_miller_indices = ['104', '110', '100', '101', '001', '012']
siderite_surface = dict.fromkeys(siderite_surfaces_miller_indices)

In [22]:
kept_surfaces = []
list_of_layers = np.linspace(5, 20, 10)
for i in list_of_layers:
    print(i)
    kept_surfaces.append(gen_surfaces(layers=i))

5.0
Layers: 5.0
6.666666666666667
Layers: 6.666666666666667
8.333333333333334
Layers: 8.333333333333334
10.0
Layers: 10.0
11.666666666666668
Layers: 11.666666666666668
13.333333333333334
Layers: 13.333333333333334
15.0
Layers: 15.0
16.666666666666668
Layers: 16.666666666666668
18.333333333333336
Layers: 18.333333333333336
20.0
Layers: 20.0


## Viewing the Results

In [2]:
ase_traj = Trajectory(r"G:\ephemeral\2371379.cx1\siderite_bulk_331.traj", 'r')